In [2]:
from pprint import pprint
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
from lib_management.models import *

In [4]:
from rest_framework.serializers import ModelSerializer

In [5]:
class ProfileSerializer(ModelSerializer):
    class Meta:
        model = Profile
        fields = '__all__'

In [6]:
obj=Profile.objects.all()
seri=ProfileSerializer(obj, many=True)
seri.data

[{'uid': '1f8053ab-3578-4383-97db-1aa12bcf5aaf', 'created_at': '2024-06-14T09:18:26.540139Z', 'updated_at': '2024-06-14T09:18:26.540154Z', 'name': 'PIYUSH GUPTA', 'email': 'piyushg094@gmail.com', 'phone': '5654324565', 'roles': 'admin'}, {'uid': '9b708302-a2b0-4e7d-b77b-ddd924738ef5', 'created_at': '2024-06-14T09:18:42.824187Z', 'updated_at': '2024-06-14T09:18:42.824204Z', 'name': 'ayush', 'email': 'ayush@gmail.com', 'phone': '3454657645', 'roles': 'user'}, {'uid': 'f3abdd38-3f65-4852-b148-9c7fe0d3bf56', 'created_at': '2024-06-14T09:33:20.785954Z', 'updated_at': '2024-06-14T09:33:20.785974Z', 'name': 'raja', 'email': 'raja@gmail.com', 'phone': '4534234567', 'roles': 'user'}]

In [7]:
class BaseModelSerializer(ModelSerializer):
    class Meta:
        abstract = True


In [8]:
class ProfileSerializer(ModelSerializer):
    class Meta:
        model = Profile
        fields = ['uid', 'name', 'email', 'phone', 'roles', 'created_at', 'updated_at']


In [9]:
class AddressSerializer(ModelSerializer):
    profile = ProfileSerializer()

    class Meta:
        model = Address
        fields = ['profile', 'city', 'state', 'line1', 'line2']
   

In [29]:
class AuthorSerializer(ModelSerializer):
    class Meta:
        model = Author
        fields = ['uid', 'name']


In [36]:
class BookDetailSerializer(ModelSerializer):
    authors = AuthorSerializer(many=True)

    class Meta:
        model = BookDetail
        fields = ['uid', 'name', 'authors', 'created_at', 'updated_at']
        
    def create(self, validated_data):
        authors_data = validated_data.pop('authors')
        book_detail = BookDetail.objects.create(**validated_data)
        for author_data in authors_data:
            author, created = Author.objects.get_or_create(name=author_data['name'])
            book_detail.authors.add(author)
        return book_detail

    def update(self, instance, validated_data):
        authors_data = validated_data.pop('authors')
        instance.name = validated_data.get('name', instance.name)
        instance.save()

        # Clear existing authors and add new ones
        instance.authors.clear()
        for author_data in authors_data:
            author, created = Author.objects.get_or_create(name=author_data['name'])
            instance.authors.add(author)
        return instance    

In [31]:
class BookRecordSerializer(ModelSerializer):
    book = BookDetailSerializer()

    class Meta:
        model = BookRecord
        fields = ['uid', 'book', 'edition', 'count', 'price', 'available', 'category', 'created_at', 'updated_at']


In [32]:
class IssueRecordSerializer(ModelSerializer):
    user = ProfileSerializer()
    book = BookDetailSerializer()

    class Meta:
        model = IssueRecord
        fields = ['uid', 'user', 'book', 'status', 'issue_date', 'return_date', 'fine', 'created_at', 'updated_at']

In [20]:
def list():
    queryset=BookDetail.objects.all()
    serializer=BookDetailSerializer(queryset,many=True)
    print(serializer.data)
    

In [21]:

list()

[{'uid': 'f2ec7f88-8bd3-432a-bb79-270199c3633c', 'name': 'jai shree ram', 'authors': [{'uid': '8406a066-7752-4bd4-a0cc-32958032f91b', 'name': 'narendra modi', 'created_at': '2024-06-14T09:28:46.659334Z', 'updated_at': '2024-06-14T09:28:46.659349Z'}, {'uid': 'cb401bea-caa8-45d0-8049-b000fb4fa121', 'name': 'sonu nigam', 'created_at': '2024-06-14T09:29:06.385741Z', 'updated_at': '2024-06-14T09:29:06.385754Z'}], 'created_at': '2024-06-14T09:29:47.876616Z', 'updated_at': '2024-06-14T09:29:47.876632Z'}, {'uid': '887965a4-1b63-425b-8d57-c169b9dead82', 'name': 'wings of fire', 'authors': [{'uid': '38fadbfc-6bd3-4822-934f-bbed923878ae', 'name': 'abdul kalam', 'created_at': '2024-06-14T09:29:57.126019Z', 'updated_at': '2024-06-14T09:29:57.126052Z'}], 'created_at': '2024-06-14T09:30:09.613092Z', 'updated_at': '2024-06-14T09:30:09.613105Z'}, {'uid': 'a78cf42a-04ac-4948-8b11-ce75a1b36ab2', 'name': 'why this kolabary', 'authors': [{'uid': 'cb401bea-caa8-45d0-8049-b000fb4fa121', 'name': 'sonu nigam',

In [46]:
def create_book_details(data):
        serializer = BookDetailSerializer(data=data)
        if serializer.is_valid():
            book_detail = serializer.save()
            return {
                "message": "Book details saved successfully",
                "data": serializer.data
            }
        return {"message": serializer.errors}

In [47]:
data = {
    "name": "see-you-again",
    "authors": [
        {"name": "ayush"},
        {"name": "charli"},
        {"name": "chirag"}
    ]
}
response=create_book_details(data);
print(response)

{'message': 'Book details saved successfully', 'data': {'uid': '0c85f9a7-4b3c-4811-82b7-db1f5d75bb3c', 'name': 'see-you-again', 'authors': [{'uid': 'dc9194fd-2c74-42da-b606-68a864bad52b', 'name': 'ayush'}, {'uid': '7f220cdc-a3a9-40ee-a2ad-83599a045a75', 'name': 'charli'}, {'uid': '333b3b7e-e765-456b-a501-1ba36ee6dd83', 'name': 'chirag'}], 'created_at': '2024-06-18T04:03:05.155501Z', 'updated_at': '2024-06-18T04:03:05.155537Z'}}


In [48]:
 def update_book_detail(book_id, data):
        try:
            book_detail = BookDetail.objects.get(uid=book_id)
        except BookDetail.DoesNotExist:
            return {"message": "BookDetail not found"}

        serializer = BookDetailSerializer(book_detail, data=data)
        if serializer.is_valid():
            serializer.save()
            return {
                "message": "Book details updated successfully",
                "data": serializer.data
            }
        return {"message": serializer.errors}

In [49]:
data = {
    "name": "see-you-again",
    "authors": [
        {"name": "ayush"},
        {"name": "charli"}
    ]
}
id='0c85f9a7-4b3c-4811-82b7-db1f5d75bb3c'
response=update_book_detail(id,data);
print(response)

{'message': 'Book details updated successfully', 'data': {'uid': '0c85f9a7-4b3c-4811-82b7-db1f5d75bb3c', 'name': 'see-you-again', 'authors': [{'uid': 'dc9194fd-2c74-42da-b606-68a864bad52b', 'name': 'ayush'}, {'uid': '7f220cdc-a3a9-40ee-a2ad-83599a045a75', 'name': 'charli'}], 'created_at': '2024-06-18T04:03:05.155501Z', 'updated_at': '2024-06-18T04:04:29.909344Z'}}


In [50]:
def retrieve_book_detail(book_id):
        try:
            book_detail = BookDetail.objects.get(uid=book_id)
        except BookDetail.DoesNotExist:
            return {"message": "BookDetail not found"}

        serializer = BookDetailSerializer(book_detail)
        return {
            "message": "Book details retrieved successfully",
            "data": serializer.data
        }

In [52]:
id='0c85f9a7-4b3c-4811-82b7-db1f5d75bb3c'
response=retrieve_book_detail(id);
print(response)

{'message': 'Book details retrieved successfully', 'data': {'uid': '0c85f9a7-4b3c-4811-82b7-db1f5d75bb3c', 'name': 'see-you-again', 'authors': [{'uid': 'dc9194fd-2c74-42da-b606-68a864bad52b', 'name': 'ayush'}, {'uid': '7f220cdc-a3a9-40ee-a2ad-83599a045a75', 'name': 'charli'}], 'created_at': '2024-06-18T04:03:05.155501Z', 'updated_at': '2024-06-18T04:04:29.909344Z'}}


In [53]:
  def delete_book_detail(book_id):
        try:
            book_detail = BookDetail.objects.get(uid=book_id)
        except BookDetail.DoesNotExist:
            return {"message": "BookDetail not found"}

        serializer = BookDetailSerializer(book_detail)
        book_detail.delete()
        return {
            "message": "Book details deleted successfully",
            "data": serializer.data
        }

In [54]:
id='dc9194fd-2c74-42da-b606-68a864bad52b'
response=delete_book_detail(id);
print(response)

{'message': 'BookDetail not found'}
